In [ ]:
!pip install patool 

     |████████████████████████████████| 81kB 2.5MB/s 


In [ ]:
import patoolib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import TensorBoard
from random import shuffle
import os
import datetime
import cv2
import matplotlib.pyplot as plt 
from tqdm import tqdm
import time

Using TensorFlow backend.


In [ ]:
keras.backend.le

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
Images_Dir = r"/content/drive/My Drive/ML/Python Plays NFS-Xception/Train Files/Images.rar"           # compressed file
patoolib.extract_archive(Images_Dir, outdir="/content")

patool: Extracting /content/drive/My Drive/ML/Python Plays NFS-Xception/Train Files/Images.rar ...
patool: running /usr/bin/unrar x -- "/content/drive/My Drive/ML/Python Plays NFS-Xception/Train Files/Images.rar"
patool:     with cwd='/content'
patool: ... /content/drive/My Drive/ML/Python Plays NFS-Xception/Train Files/Images.rar extracted to `/content'.


'/content'

In [ ]:
Final_Data = None
Data_Piecies = {}
Data_Dir= r"/content/drive/My Drive/ML/Python Plays NFS-Xception/Train Files/Labels/Labels-part{}.pkl"
Part_Count = 6

for part_number in tqdm(range(0, Part_Count + 1)):
  file = pd.read_pickle(Data_Dir.format(part_number))
  file["Address"] = np.array([part_number for i in range(0, len(file))])
  Data_Piecies[part_number] = file

Final_Data = pd.concat(Data_Piecies)
Final_Data.index = np.arange(0, len(Final_Data))
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
print(Final_Data.tail())

100%|██████████| 7/7 [00:00<00:00, 32.30it/s]

          ID      Label  Address
73570  11525  [0, 1, 0]        6
73571  11526  [0, 0, 1]        6
73572  11527  [0, 1, 0]        6
73573  11528  [0, 1, 0]        6
73574  11529  [1, 0, 0]        6


In [ ]:
# Split Data
Validation_Size = 0.022

Place = round(len(Final_Data) * Validation_Size)

Validation_Data = Final_Data[: Place]
Train_Data = Final_Data[Place: ]

Train_Data.index = np.arange(0, len(Train_Data))
Validation_Data.index = np.arange(0, len(Validation_Data))

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print('\nTrain Data\n', Train_Data.tail())
print('\nValidation Data\n', Validation_Data.tail())


Train Data
           ID      Label  Address
71951  11525  [0, 1, 0]        6
71952  11526  [0, 0, 1]        6
71953  11527  [0, 1, 0]        6
71954  11528  [0, 1, 0]        6
71955  11529  [1, 0, 0]        6

Validation Data
         ID      Label  Address
1614  1614  [0, 1, 0]        0
1615  1615  [1, 0, 0]        0
1616  1616  [0, 0, 1]        0
1617  1617  [1, 0, 0]        0
1618  1618  [1, 0, 0]        0


In [ ]:
Image_Height = 184
Image_Weight = 299

In [ ]:
def normaliz_test_data(data):
  images, labels = [], []

  for i in tqdm(range(0, len(data))):
    Address, ID, label = data["Address"][i], data["ID"][i], data["Label"][i]
    image = cv2.imread("/content/images/{}".format(str(Address) + "/" + str(ID) + ".jpg"), 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(image)
    image = keras.applications.resnet_v2.preprocess_input(image)
    images.append(image)
    labels.append(label)

  return (np.array(images, dtype='float32'), np.array(labels, dtype='float32'))

In [ ]:
Validation_Data = normaliz_test_data(Validation_Data)

100%|██████████| 1619/1619 [00:15<00:00, 104.37it/s]


In [ ]:
def train_data_generator(data, batch_size=32):
  images, labels = [], []

  while True:
    for i in range(len(data)):
      Address, ID, label = data["Address"][i], data["ID"][i], data["Label"][i]
      image = cv2.imread("/content/images/{}".format((str(Address) + "/" + str(ID) + ".jpg")), 1)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = np.array(image, dtype="float32")
      image = keras.applications.resnet_v2.preprocess_input(image)
      images.append(image)
      labels.append(label)

      if len(images) >= batch_size:
        yield np.array(images, dtype='float32'), np.array(labels, dtype='float32')
        images, labels = [], []

In [ ]:
Epochs = 21
Batch_Size = 32
TB_LogDir = "/content/drive/My Drive/ML/Python Plays NFS-Xception/logs/{}".format(("PPN-25-epochs-Resnet152V2-ver=0.4-"+ str(datetime.datetime.now())))

In [ ]:
Train_Generator = train_data_generator(Train_Data, batch_size=Batch_Size)
TB_CallBack = TensorBoard(log_dir=TB_LogDir, histogram_freq=1)
tf.test.gpu_device_name()keras.

'/device:GPU:0'

In [ ]:
Model = keras.Sequential()
Xception = keras.applications.ResNet152V2(include_top=False, weights="imagenet", input_shape=(Image_Height, Image_Weight, 3))
Model.add(Xception)
Model.add(keras.layers.Flatten())
Model.add(keras.layers.Dense(128, activation="relu"))
Model.add(keras.layers.Dense(3, activation="softmax"))
"""
x = keras.layers.Flatten()(Xception)
x = keras.layers.Dense(1024, activation="relu")(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)
Model = keras.Model(inputs=Xception.input, outputs=predictions)"""
Model.summary()

Model.compile(loss='categorical_crossentropy', optimizer="SGD", metrics=['accuracy'])
Model.Name="Resnet152V2-25epochs"

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Model)          (None, 6, 10, 2048)       58331648  
_________________________________________________________________
flatten_1 (Flatten)          (None, 122880)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               15728768  
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 74,060,803
Trainable params: 73,917,059
Non-trainable params: 143,744
_________________________________________________________________


In [ ]:
Model.fit(Train_Generator, steps_per_epoch=len(Train_Data)/Batch_Size, epochs=Epochs, validation_data=Validation_Data, verbose=1, callbacks=[TB_CallBack])

Epoch 1/21
2249/2248 [==============================] - 1089s 484ms/step - loss: 0.7232 - accuracy: 0.6684 - val_loss: 0.6618 - val_accuracy: 0.7029
Epoch 2/21
2249/2248 [==============================] - 1084s 482ms/step - loss: 0.5704 - accuracy: 0.7410 - val_loss: 0.6815 - val_accuracy: 0.6955
Epoch 3/21
2249/2248 [==============================] - 1084s 482ms/step - loss: 0.4754 - accuracy: 0.7887 - val_loss: 0.7525 - val_accuracy: 0.7048
Epoch 4/21
2249/2248 [==============================] - 1083s 481ms/step - loss: 0.4005 - accuracy: 0.8243 - val_loss: 0.7665 - val_accuracy: 0.7103
Epoch 5/21
2249/2248 [==============================] - 1083s 482ms/step - loss: 0.3357 - accuracy: 0.8566 - val_loss: 0.8313 - val_accuracy: 0.7004
Epoch 6/21
2249/2248 [==============================] - 1083s 482ms/step - loss: 0.2727 - accuracy: 0.8872 - val_loss: 1.0184 - val_accuracy: 0.6998
Epoch 7/21
2249/2248 [==============================] - 1083s 482ms/step - loss: 0.2217 - accuracy: 0.9119

In [ ]:
print("Saving...")
Model.save(r"/content/drive/My Drive/ML/Python Plays NFS-Xception/log_ver {}-25 epochs-Resnet152V2-Python-Plays-NFS.h5".format(str(0.2)))        # Save to drive
print("Done!") 

Saving...
Done!


In [ ]:
#Model = keras.models.load_model(r"/content/drive/My Drive/ML/Python Plays NFS-Xception/ver 0.1-Xception-flatten-1Dense-relu-val_acc:0.65,train_acc:0.86-Python-Plays-NFS.h5")
img = cv2.imread("/8.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.array(img, dtype="float32")
img = keras.applications.xception.preprocess_input(img)
img = img.reshape(1, 184, 299, 3)
print(np.argmax(Model.predict(img)))

ValueError: ignored

In [ ]:
MobileNetV2 = keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(Image_Height, Image_Weight, 3))

for layer in MobileNetV2.layers:
  layer.trainable = False

Model = keras.Sequential()
Model.add(MobileNetV2)
Model.add(keras.layers.Flatten())
Model.add(keras.layers.Dense(1024, activation='relu'))
Model.add(keras.layers.Dense(512, activation='relu'))
Model.add(keras.layers.Dropout(0.2))
Model.add(keras.layers.Dense(128, activation='relu'))
Model.add(keras.layers.Dense(3, activation='softmax'))

Model.summary()
Model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 6, 10, 1280)       2257984   
_________________________________________________________________
flatten (Flatten)            (None, 76800)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              78644224  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
___________________________________________________________

In [ ]:
Model.fit(Train_Generator, steps_per_epoch=len(Train_Data)/Batch_Size, epochs=Epochs, validation_data=Validation_Data, verbose=1,  callbacks=[TB_CallBack])

Epoch 1/20
2254/2253 [==============================] - 363s 161ms/step - loss: 1.1755 - accuracy: 0.5680 - val_loss: 0.7819 - val_accuracy: 0.6393
Epoch 2/20
2254/2253 [==============================] - 368s 163ms/step - loss: 0.7682 - accuracy: 0.6422 - val_loss: 0.7993 - val_accuracy: 0.6433
Epoch 3/20
 139/2253 [>.............................] - ETA: 5:17 - loss: 0.7272 - accuracy: 0.6603

KeyboardInterrupt: ignored

In [ ]:
print("Saving...")
Model.save(r"/content/drive/My Drive/ML/Python Plays NFS-Xception/ver {}-MobileNetV2-Python-Plays-NFS.h5".format(str(0.3)))        # Save to drive
Model.save(ModelName)
print("Done!") 

In [ ]:
InceptionV3 = keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(Image_Height, Image_Weight, 3))

for layer in InceptionV3.layers:
  layer.trainable = False

Model = keras.Sequential()
Model.add(InceptionV3)
Model.add(keras.layers.Flatten())
Model.add(keras.layers.Dense(1024, activation='relu'))
Model.add(keras.layers.Dense(512, activation='relu'))
Model.add(keras.layers.Dropout(0.2))
Model.add(keras.layers.Dense(128, activation='relu'))
Model.add(keras.layers.Dense(3, activation='softmax'))

Model.summary()
Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

87916544/87910968 [==============================] - 1s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 4, 8, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              67109888  
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               65664     
_______________________________________________________

In [ ]:
Model.fit(Train_Generator, steps_per_epoch=len(Train_Data)/Batch_Size, epochs=Epochs, validation_data=Validation_Data, verbose=1,  callbacks=[TB_CallBack])

Epoch 1/20
 125/2253 [>.............................] - ETA: 9:11 - loss: 7.5831 - accuracy: 0.3830

KeyboardInterrupt: ignored

In [ ]:
print("Saving...")
Model.save(r"/content/drive/My Drive/ML/Python Plays NFS-Xception/ver {}-InceptionV3-Python-Plays-NFS.h5".format(str(0.5)))        # Save to drive
Model.save(ModelName)
print("Done!") 